In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Demander à l'utilisateur de fournir la clé OpenAI
OPENAI_API_KEY = ""

# Le nom du fichier d'entrée. Ce doit être un csv. Le séparateur est normalement détecté automatiquement.
filename = "/content/drive/MyDrive/Entretiens_Extraction.csv"

# Le chemin du fichier CSV où sauvegarder le résultat
output_filename = "/content/drive/MyDrive/Entretiens_Extraction_GPT_response.csv"

# Le nom de la colonne de texte dans le fichier d'entrée
text_column = "TEXTE"

# Nom ou identifiant du modèle GPT-4 à utiliser lors des requêtes à l'API OpenAI.
model_name = "gpt-4o-mini"

# Prompt initial fourni au système. Ce texte décrit le contexte et les instructions que le modèle doit suivre. Pour l'exemple ici :
# 1) Attribuer une note au message (score) de 0 à 10 selon les critères d'objectivité et de désinformation.
# 2) Classifier le message dans l'une des trois catégories politiques définies.
system_expected = """
Analysez le message suivant et exécutez les deux tâches suivantes, dans cet ordre :

Première tâche : attribuer une note entre 0 et 10
Attribuez un nombre entier compris entre 0 et 10 au message, selon les critères suivants :
- 0 : Le message est entièrement basé sur des faits véridiques et vérifiés, sans aucun élément de subjectivité ou d'erreur.
- 1 à 4 : Le message contient des éléments subjectifs, mais reste globalement neutre ou vrai. Donnez 5 si le message est ironique (l'ironie n'est pas trompeuse, mais seulement très subjective).
- 5 à 7 : Le message contient des éléments qui peuvent induire en erreur, sans être totalement faux. Par exemple, des affirmations partielles ou des simplifications exagérées.
- 8 à 10 : Le message semble délibérément trompeur ou complètement faux, avec des affirmations contraires à des faits établis.

IMPORTANT : Retournez uniquement le nombre entier correspondant à cette évaluation. Sans explications ou textes supplémentaires.

Deuxième tâche : catégoriser politiquement le message
Classifiez le message dans l'une des trois catégories suivantes :
- PRO-TRAVAILLISTE : Le message présente une vision favorable à la gauche travailliste ou contient des critiques explicites du bolsonarisme.
- PRO-BOLSONARO : Le message présente une vision favorable au bolsonarisme ou contient des critiques explicites de la gauche travailliste.
- AMBIGU OU INDÉTERMINÉ : Le message ne contient pas d'éléments clairs, suffisants ou explicites permettant de l'associer à l'une des deux catégories précédentes.

Ne catégorisez pas comme travailliste ou bolsonariste les messages présentant des doutes ou des ambiguïtés. Seuls les messages qui présentent des critiques explicites envers l'autre camp, ou une défense de positions très polarisantes et très significatives pour l'un des deux camps, doivent être classés dans ces catégories.

IMPORTANT : Retournez uniquement la catégorie exacte, et rien d'autre : "PRO-TRAVAILLISTE", "PRO-BOLSONARO" ou "AMBIGU OU INDÉTERMINÉ".
"""

# Structure de la réponse attendue au format JSON. Elle impose un schéma précis :
# un objet JSON contenant un entier "score" et une chaîne "category" devant être l'une des trois valeurs autorisées.
# Ce schéma permet de valider la forme et le contenu de la réponse retournée par le modèle.
response_expected = {
    "type": "json_schema",
    "json_schema": {
        "name": "classification_task",
        "schema": {
            "type": "object",
            "properties": {
                "score": {"type": "integer"},
                "category": {
                    "type": "string",
                    "enum": ["PRO-TRAVAILLISTE", "PRO-BOLSONARO", "AMBIGU OU INDÉTERMINÉ"]
                }
            },
            "required": ["score", "category"],
            "additionalProperties": False
        },
        "strict": True
    }
}

In [ ]:
%%capture
!pip install --upgrade openai

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import os
from tqdm.notebook import tqdm
from IPython.display import clear_output
import time
import json
import openai
import csv
openai.api_key = OPENAI_API_KEY

In [ ]:
client = openai

In [ ]:
def classifier_gpt4(message):
    max_retries = 3
    retry_delay = 5  # secondes

    for attempt in range(max_retries):
        try:
            # Effectuer la requête
            response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {
                        "role": "system",
                        "content": system_expected
                    },
                    {"role": "user", "content": message}
                ],
                response_format=response_expected
            )

            # Vérifier si le contenu est bien du JSON
            response_content = response.choices[0].message.content

            print('---- ' + str(response_content) + ' ---- ' + message)

            if isinstance(response_content, str):
                try:
                    structured_response = json.loads(response_content)  # Décoder le JSON
                except json.JSONDecodeError:
                    raise ValueError("La réponse n'est pas au format JSON attendu")
            else:
                structured_response = response_content

            # Valider les champs
            score = structured_response.get("score")
            category = structured_response.get("category")

            # Validation manuelle
            if not isinstance(score, int) or not (0 <= score <= 10):
                raise ValueError("Score invalide")
            if category not in ["PRO-TRAVAILLISTE", "PRO-BOLSONARO", "AMBIGU OU INDÉTERMINÉ"]:
                raise ValueError("Catégorie invalide")

            return {"score": score, "category": category}

        except Exception as e:
            print(f"Erreur API ou autre : {e}. Nouvelle tentative dans {retry_delay} secondes...")
            time.sleep(retry_delay)

    return "Error"

In [ ]:
def read_csv_with_auto_separator(file_path, text_col):
    # Lecture directe du CSV avec le séparateur ';'
    df = pd.read_csv(file_path, sep=';', low_memory=False)
    # Vérifier la présence de la colonne texte
    if text_col not in df.columns:
        raise ValueError(f"La colonne '{text_col}' n'existe pas dans le fichier CSV.")
    return df

# Lecture du fichier source sans détection automatique (nous savons que c'est ';')
df = read_csv_with_auto_separator(filename, text_column)

# Ajouter une colonne 'index' si elle n'existe pas
if 'index' not in df.columns:
    df.reset_index(inplace=True)

# Exemple d'appel au classificateur sur un message vide pour connaître la structure retournée.
# On suppose ici que classifier_gpt4 retourne toujours un dict avec les clés prédictives.
sample_label = classifier_gpt4("Message factice pour tester la structure")
prediction_columns = list(sample_label.keys())  # Extraire les noms de colonnes depuis le retour du classificateur
# Exemple : prediction_columns = ['score', 'category'] s'il retourne {"score": 3, "category": "AMBIGU OU INDÉTERMINÉ"}

# Ajouter le préfixe 'predicted_' pour différencier les colonnes source des colonnes prédictives
prediction_columns = [f"predicted_{col}" for col in prediction_columns]

if os.path.exists(output_filename):
    print("Reprise à partir d'une sauvegarde existante.")
    df_existing = pd.read_csv(output_filename, sep=';')
    required_columns = ['index'] + prediction_columns
    if not all(col in df_existing.columns for col in required_columns):
        missing_cols = [col for col in required_columns if col not in df_existing.columns]
        raise ValueError(f"Le fichier de sauvegarde existant ne contient pas toutes les colonnes requises : {missing_cols}")
    df = df.merge(df_existing[required_columns], on='index', how='left')
else:
    # Initialiser les colonnes de prédiction à None s'il n'y a pas de sauvegarde
    for col in prediction_columns:
        if col not in df.columns:
            df[col] = None

# Filtrer les messages non encore classifiés
df_to_process = df[df[prediction_columns].isnull().all(axis=1)]

# Traitement et suivi avec tqdm
for idx, (i, row) in enumerate(tqdm(df_to_process.iterrows(), total=len(df_to_process), desc="Traitement des messages")):
    message = row[text_column]
    label = classifier_gpt4(message)

    if isinstance(label, dict):
        # Mettre à jour toutes les colonnes prédictives de manière générique
        for col in prediction_columns:
            # Extraire le nom originel sans le préfixe 'predicted_'
            original_key = col.replace('predicted_', '')
            df.loc[df['index'] == row['index'], col] = label.get(original_key, None)
        # Affichage (optionnel, à adapter si besoin)
        print("Prédictions :")
        for col in prediction_columns:
            print(f"{col} : {df.loc[df['index'] == row['index'], col].values[0]}")
    else:
        # Si label n'est pas un dict, mettre None partout (optionnel)
        for col in prediction_columns:
            df.loc[df['index'] == row['index'], col] = None

    # Sauvegarde incrémentielle toutes les 1000 itérations
    if idx > 0 and idx % 1000 == 0:
        df.to_csv(output_filename, index=False, encoding='utf-8', sep=';')
        print(f"Progrès sauvegardé à {idx} messages traités.")

# Sauvegarde finale
df.to_csv(output_filename, index=False, encoding='utf-8', sep=';')